<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/LSTM_with_Pytorch_Recarp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f"You are on Google CoLaB with Pytorch version: {torch.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n>>>please corect {type(e)} and reload")
  COLAB = False
def time_fmt(t: float = 123.982)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print(f">>>time formating:\tplease wait....\n>>>time elapse: \t{time_fmt()}")

Mounted at /content/drive
You are on Google CoLaB with Pytorch version: 1.8.1+cu101
>>>time formating:	please wait....
>>>time elapse: 	0: 02: 03.00


In [2]:
#In this notebook we are going to train an simple rnn with an LSTM architecture to 
#train on mnist data(treated as a sequence data for demo only)
#We take heght and width of mnist images as sequence lenght and deature size respectivelly
#Since mnist has the shape of (1, 28,28) will squeeze the channel dimension

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from tqdm import tqdm
import time, sys, datetime

In [4]:
#Hyperparameters 
batch_size = 64
EPOCHS = 10
learning_rate = 1e-3
input_size = 28
sequence_length = 28
num_classes = 10
num_layers = 2
hidden_size = 512

In [5]:
#The LSTM class: This class will build an rnn with 2 LSTM layer and a final output FC layer

In [6]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers,num_classes):
    super(LSTM, self).__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.lstm = nn.LSTM(input_size,hidden_size,num_layers,batch_first = True)
    self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
  
  def forward(self, input_tensor):
    #initialize cell state and hidden state to zeros
    h0 = torch.zeros(self.num_layers, input_tensor.size(0), self.hidden_size).to(device = device)
    c0 = torch.zeros(self.num_layers, input_tensor.size(0), self.hidden_size).to(device = device)
    output,_ = self.lstm(input_tensor, (h0,c0))
    output = output.reshape(output.shape[0],-1)
    return self.fc(output)
    

In [7]:
#Instantiating and testing the model class to reveal output shape (64,10)?

In [9]:
model = LSTM(input_size, hidden_size, num_layers,num_classes).to(device = device)
rnd_data = torch.rand(64, 28,28).to(device = device)
print(f"output shape: {model(rnd_data).shape}")

output shape: torch.Size([64, 10])


In [10]:
#load the data from torch:
train_data = datasets.MNIST(root = '/train_data', train = True, transform = transforms.ToTensor(), download = True)
test_data = datasets.MNIST(root = '/test_data', train = False,transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_data, shuffle = True, batch_size = batch_size)
test_loader = DataLoader(dataset = test_data, shuffle = True, batch_size = batch_size)
x_batch, y_batch = next(iter(train_loader))
print(f"x_batch_shape: {x_batch.shape}\ty_batch_shape:{y_batch.shape}")


Extracting /train_data/MNIST/raw/train-images-idx3-ubyte.gz to /train_data/MNIST/raw




Extracting /train_data/MNIST/raw/train-labels-idx1-ubyte.gz to /train_data/MNIST/raw




Extracting /train_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /train_data/MNIST/raw




Extracting /train_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /train_data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)



Extracting /test_data/MNIST/raw/train-images-idx3-ubyte.gz to /test_data/MNIST/raw




Extracting /test_data/MNIST/raw/train-labels-idx1-ubyte.gz to /test_data/MNIST/raw




Extracting /test_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /test_data/MNIST/raw




Extracting /test_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /test_data/MNIST/raw

Processing...
Done!
x_batch_shape: torch.Size([64, 1, 28, 28])	y_batch_shape:torch.Size([64])


In [11]:
#Get optimizer and loss object
loss_obj = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = model.parameters(), lr = learning_rate)


In [12]:
#Training loop from the scratch:
tic = time.time()
for epoch in range(EPOCHS):
  print(f"\n>>>Training start for epoch {epoch + 1}\n>>>please wait while the model is training!!!\tMAINTAIN SCREEN ACTIVITY!!")
  for idx, (data, target) in enumerate(tqdm(train_loader)):
    #load data to gpu/cpu
    data = data.to(device = device).squeeze(1)
    target = target.to(device = device)
    #Forward pass
    preds = model(data)
    train_loss = loss_obj(preds, target)
    #Backward pass
    optimizer.zero_grad()
    train_loss.backward()
    #gradient descent with Adam
    optimizer.step()
#Tracking Model performance:
def model_performance(model, loader):
  if loader.dataset.train:
    print(f"\n>>>>Working on the training dataset:\nPlease wait...\tMAINTAIN SCREEN ACTIVITY") 
  else:
    print(f"\n>>>Working on the validation dataset\nPlease wait...\tMAINTAIN SCREEN ACTIVITY")
  num_examples = 0
  num_correct = 0
  model.eval()
  #No need to compute the slope again
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device = device).squeeze(1)
      y = y.to(device = device)
      preds = model(x)
      _, predictions = preds.max(1)
      num_correct+=(predictions == y).sum()
      num_examples+=predictions.size(0)
  model.train()
  return float(num_correct/num_examples)*100
toc = time.time()
print(f"\n>>>Accuracy on the training data: {model_performance(model, train_loader):.2f}")
print(f"\n>>>Accuracy on the validation data: {model_performance(model, test_loader):.2f}")
print(f"\n>>>Total time for training and evaluation is: {time_fmt(toc - tic)}")



  0%|          | 1/938 [00:00<01:59,  7.83it/s]


>>>Training start for epoch 1
>>>please wait while the model is training!!!	MAINTAIN SCREEN ACTIVITY!!


  1%|          | 8/938 [00:00<00:11, 79.19it/s]


>>>Training start for epoch 2
>>>please wait while the model is training!!!	MAINTAIN SCREEN ACTIVITY!!


  1%|          | 8/938 [00:00<00:12, 75.67it/s]

...


>>>Training start for epoch 9
>>>please wait while the model is training!!!	MAINTAIN SCREEN ACTIVITY!!


  1%|          | 8/938 [00:00<00:12, 73.92it/s]


>>>Training start for epoch 10
>>>please wait while the model is training!!!	MAINTAIN SCREEN ACTIVITY!!


100%|██████████| 938/938 [00:13<00:00, 67.37it/s]



>>>>Working on the training dataset:
Please wait...	MAINTAIN SCREEN ACTIVITY

>>>Accuracy on the training data: 99.64

>>>Working on the validation dataset
Please wait...	MAINTAIN SCREEN ACTIVITY

>>>Accuracy on the validation data: 99.25

>>>Total time for training and evaluation is: 0: 02: 16.00
